In [1]:
from Idntimes import Idntimes
from datetime import timedelta, date, datetime
from IPython.display import clear_output

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
obj = Idntimes()

In [2]:
end_date = date(2018, 8, 12)
start_date = date(2018, 5, 18)
init_page = 1

In [3]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], 'news', init_page, single_date.strftime("%Y-%m-%d"))
    init_page = 1

2018-05-27
page  1
https://www.idntimes.com/ajax/index?category=news&type=all&page=1&date=2018-05-27
https://www.idntimes.com/news/world/andes-3/buntut-video-penistaan-nabi-muhammad-mesir-blokir-youtube-c1c2
memasukkan berita id  1527354106
Insert berita  Buntut Video Penistaan Nabi Muhammad, Mesir Blokir YouTube Selama Sebulan
masuk
https://www.idntimes.com/news/world/yakobus-david-lay-se/pm-shinzo-abe-gagal-bujuk-presiden-rusia-soal-sengketa-kepulauan-kuril-c1c2
memasukkan berita id  1527354132
Insert berita  PM Shinzo Abe Gagal Bujuk Presiden Rusia Soal Sengketa Kepulauan Kuril
masuk
https://www.idntimes.com/news/world/karl-gading-s/turki-kutuk-keputusan-jerman-izinkan-partai-pro-kurdi-kampanye-c1c2
memasukkan berita id  1527354117
Insert berita  Turki Kutuk Keputusan Jerman Izinkan Partai Pro-Kurdi Kampanye
masuk
https://www.idntimes.com/news/indonesia/indianamalia/gaji-dewan-bpip-jadi-sorotan-maki-siap-gugat-perpres-no
memasukkan berita id  1527354108
Insert berita  Gaji Dewan BPI

KeyboardInterrupt: 